# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 9 2022 12:45PM,250417,19,ACG-2102491811,ACG North America LLC,Released
1,Nov 9 2022 12:43PM,250416,15,PNC384552B,"Person & Covey, Inc.",Released
2,Nov 9 2022 12:43PM,250416,15,PNC401220A,"Person & Covey, Inc.",Released
3,Nov 9 2022 12:43PM,250416,15,PNC401325A,"Person & Covey, Inc.",Released
4,Nov 9 2022 12:43PM,250416,15,PNC401338A,"Person & Covey, Inc.",Released
5,Nov 9 2022 12:43PM,250416,15,PNC401339A,"Person & Covey, Inc.",Released
6,Nov 9 2022 12:43PM,250416,15,PNC401340A,"Person & Covey, Inc.",Released
7,Nov 9 2022 12:43PM,250416,15,PNC401341A,"Person & Covey, Inc.",Released
8,Nov 9 2022 12:43PM,250416,15,PNC401349A,"Person & Covey, Inc.",Released
9,Nov 9 2022 12:43PM,250416,15,PNC401355A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,250411,Released,8
30,250413,Executing,11
31,250413,Released,18
32,250416,Released,83
33,250417,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250410,NaN,NaN,1.0
250411,NaN,5.0,8.0
250413,NaN,11.0,18.0
250416,NaN,NaN,83.0
250417,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250318,73.0,8.0,9.0
250338,0.0,0.0,1.0
250356,0.0,0.0,1.0
250357,0.0,0.0,1.0
250359,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250318,73,8,9
250338,0,0,1
250356,0,0,1
250357,0,0,1
250359,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250318,73,8,9
1,250338,0,0,1
2,250356,0,0,1
3,250357,0,0,1
4,250359,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250318,73,8,9
1,250338,,,1
2,250356,,,1
3,250357,,,1
4,250359,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc."
84,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd"
113,Nov 9 2022 12:11PM,250411,10,Bio-PRF
126,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC"
127,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc."
128,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc."
129,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.
146,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.
149,Nov 9 2022 10:26AM,250382,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC,,,1
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",,,83
2,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd",,11,18
3,Nov 9 2022 12:11PM,250411,10,Bio-PRF,,5,8
4,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",,,1
5,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",,,1
6,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",,,1
7,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.,3,5,9
8,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.,,,3
9,Nov 9 2022 10:26AM,250382,10,Bio-PRF,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC,1,,
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83,,
2,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd",18,11,
3,Nov 9 2022 12:11PM,250411,10,Bio-PRF,8,5,
4,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",1,,
5,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1,,
6,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",1,,
7,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.,9,5,3
8,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.,3,,
9,Nov 9 2022 10:26AM,250382,10,Bio-PRF,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC,1,,
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83,,
2,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd",18,11,
3,Nov 9 2022 12:11PM,250411,10,Bio-PRF,8,5,
4,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC,1.0,NaN,NaN
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83.0,NaN,NaN
2,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd",18.0,11.0,NaN
3,Nov 9 2022 12:11PM,250411,10,Bio-PRF,8.0,5.0,NaN
4,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 12:45PM,250417,19,ACG North America LLC,1.0,0.0,0.0
1,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83.0,0.0,0.0
2,Nov 9 2022 12:13PM,250413,19,"AdvaGen Pharma, Ltd",18.0,11.0,0.0
3,Nov 9 2022 12:11PM,250411,10,Bio-PRF,8.0,5.0,0.0
4,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2003142,46.0,14.0,19.0
15,751098,102.0,8.0,77.0
16,1001461,4.0,0.0,0.0
19,500830,19.0,11.0,0.0
21,2003070,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2003142,46.0,14.0,19.0
1,15,751098,102.0,8.0,77.0
2,16,1001461,4.0,0.0,0.0
3,19,500830,19.0,11.0,0.0
4,21,2003070,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,14.0,19.0
1,15,102.0,8.0,77.0
2,16,4.0,0.0,0.0
3,19,19.0,11.0,0.0
4,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,15,Released,102.0
2,16,Released,4.0
3,19,Released,19.0
4,21,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,19.0,77.0,0.0,0.0,0.0
Executing,14.0,8.0,0.0,11.0,1.0
Released,46.0,102.0,4.0,19.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,19.0,77.0,0.0,0.0,0.0
1,Executing,14.0,8.0,0.0,11.0,1.0
2,Released,46.0,102.0,4.0,19.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,19.0,77.0,0.0,0.0,0.0
1,Executing,14.0,8.0,0.0,11.0,1.0
2,Released,46.0,102.0,4.0,19.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()